In [1]:
import torch.nn as nn
import torchvision
from torchvision.models import resnet50, resnet18, squeezenet1_1
from torchvision.models.resnet import ResNet50_Weights, ResNet18_Weights
from torchvision.models import efficientnet_v2_l
from torchvision.models.efficientnet import EfficientNet_V2_L_Weights
from torchvision.models import vgg16, vgg19
from torchvision.models.vgg import VGG16_Weights, VGG19_Weights
from torchvision.models.squeezenet import SqueezeNet1_1_Weights


import glob
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
class LaBocciaModel(nn.Module):
    def __init__(self):
        super(LaBocciaModel, self).__init__()
        #self.pretrained = squeezenet1_1(weights = SqueezeNet1_1_Weights.DEFAULT)
        #self.pretrained = resnet50(weights = ResNet50_Weights.DEFAULT).eval()
        #self.pretrained = vgg16(weights = VGG16_Weights.DEFAULT).eval()
        #self.pretrained = vgg19(weights = VGG19_Weights.DEFAULT).eval()
        #self.pretrained = resnet18(weights = ResNet18_Weights.DEFAULT).eval()
        self.pretrained = efficientnet_v2_l(weights = EfficientNet_V2_L_Weights.DEFAULT).eval()
        self.pretrained.fc = nn.Identity()

    def forward(self, x):
        x = self.pretrained(x)
        return x

In [3]:
from deepface.commons import functions

input_dir = "/home/disi/Project-IML/query/"
output_dir = "/home/disi/Project-IML/aligned_query/"

for file_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    detection = functions.extract_faces(img = img, enforce_detection=False)
    x, y, w, h = detection[0][1].values()
    aligned_img = img[int(y):int(y+h), int(x):int(x+w)]
    aligned_img = cv2.cvtColor(aligned_img, cv2.COLOR_BGRA2RGB)
    aligned_img = cv2.resize(aligned_img, (160, 160))
    cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)

input_dir = "/home/disi/Project-IML/gallery/"
output_dir = "/home/disi/Project-IML/aligned_gallery/"

for file_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    detection = functions.extract_faces(img = img, enforce_detection=False)
    x, y, w, h = detection[0][1].values()
    aligned_img = img[int(y):int(y+h), int(x):int(x+w)]
    aligned_img = cv2.cvtColor(aligned_img, cv2.COLOR_BGRA2RGB)
    aligned_img = cv2.resize(aligned_img, (160, 160))
    cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)

2023-05-15 13:39:48.936109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 13:39:57.453217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import pickle

data_dir = "/home/disi/Project-IML/aligned_gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = LaBocciaModel()
model.to(device)

embeddings = {}

for path in gallery_paths:
    image = cv2.imread(os.path.join(data_dir, path))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    #image = cv2.normalize(np.asarray(image), None, 0, 1.0, cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #image = Image.fromarray(image.astype(np.uint8))
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160), antialias=True)(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5,0.5), std=(0.5, 0.5, 0.5))(image)

    image = image.unsqueeze(0)
    image = image.to(device)
    embeddings[path] = model(image).detach().cpu().numpy()

# save embeddings to disk
with open("/home/disi/Project-IML/embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

data_dir = "/home/disi/Project-IML/aligned_query/"
queries = {}
query_paths = glob.glob(f"{data_dir}*", recursive=True)
for query in query_paths:
    image = cv2.imread(os.path.join(data_dir, query))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    #image = cv2.normalize(np.asarray(image), None, 0, 1.0, cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #image = Image.fromarray(image.astype(np.uint8))
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160), antialias=True)(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    queries[query] = model(image).detach().cpu().numpy()

In [5]:
def euclidian_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

def extract_id(string):
    splitter = string.split('/')
    splitter = splitter[-1].split('_')
    splitter = splitter[0]

    return splitter

def number_equal(sort):
    query_num = 0

    for i in sort:
        i_id = extract_id(i[0])
        if i_id == query_id:
            query_num += 1

    return query_num

query_num_list_10 = []
query_num_list_5 = []
query_num_list_1 = []

for query in queries:
    query_embedding = queries[query]
    distances = {}
    counter = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

        gallery_id = extract_id(gallery)
        if gallery_id in counter:
            counter[gallery_id] += 1
        else:
            counter[gallery_id] = 1

    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    #print(f"Query: {query}")
    #print(sorted_distances[:5])
    sorted_distances_10 = sorted_distances[:10]
    sorted_distances_5 = sorted_distances[:5]
    sorted_distances_1 = sorted_distances[:1]

    query_id = extract_id(query)

    query_num_10 = number_equal(sorted_distances_10)
    query_num_5 = number_equal(sorted_distances_5)
    query_num_1 = number_equal(sorted_distances_1)

    query_num_list_10.append(query_num_10)
    query_num_list_5.append(query_num_5)
    query_num_list_1.append(query_num_1)
    #print(f'Gallery: {counter[query_id]}, found: {query_num}')

print(f'Raw accuracy top 10 euclidian distance: {np.array(query_num_list_10).mean()/np.array(list(counter.values())).mean()}')
print(f'Raw accuracy top 5 euclidian distance: {np.array(query_num_list_5).mean()/min(np.array(list(counter.values())).mean(),5)}')
print(f'Raw accuracy top 1 euclidian distance: {np.array(query_num_list_1).mean()/min(np.array(list(counter.values())).mean(),1)}')

# now again but with another distance metric

from sklearn.metrics.pairwise import cosine_similarity


query_num_list_10 = []
query_num_5 = []
query_num_1 = []
for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
    #print(f"Query: {query}")
    #print(sorted_distances[:5])

    sorted_distances_10 = sorted_distances[:10]
    sorted_distances_5 = sorted_distances[:5]
    sorted_distances_1 = sorted_distances[:1]

    query_id = extract_id(query)

    query_num_10 = number_equal(sorted_distances_10)
    query_num_5 = number_equal(sorted_distances_5)
    query_num_1 = number_equal(sorted_distances_1)

    query_num_list_10.append(query_num_10)
    query_num_list_5.append(query_num_5)
    query_num_list_1.append(query_num_1)
    #print(f'Gallery: {counter[query_id]}, found: {query_num}')

print(f'Raw accuracy top 10 cosine similarity: {np.array(query_num_list_10).mean()/np.array(list(counter.values())).mean()}')
print(f'Raw accuracy top 5 cosine similarity: {np.array(query_num_list_5).mean()/min(np.array(list(counter.values())).mean(),5)}')
print(f'Raw accuracy top 1 cosine similarity: {np.array(query_num_list_1).mean()/min(np.array(list(counter.values())).mean(),1)}')


Raw accuracy top 10 euclidian distance: 0.6037735849056605
Raw accuracy top 5 euclidian distance: 0.52
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.6698113207547169
Raw accuracy top 5 cosine similarity: 0.525
Raw accuracy top 1 cosine similarity: 1.0


EfficientNet:

Raw accuracy top 10 euclidian distance: 0.5943396226415094
Raw accuracy top 5 euclidian distance: 0.51
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.6603773584905661
Raw accuracy top 5 cosine similarity: 0.52
Raw accuracy top 1 cosine similarity: 1.0

VGG19

Raw accuracy top 10 euclidian distance: 0.37735849056603776
Raw accuracy top 5 euclidian distance: 0.26
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.44339622641509435
Raw accuracy top 5 cosine similarity: 0.27999999999999997
Raw accuracy top 1 cosine similarity: 1.0

Resnet50

Raw accuracy top 10 euclidian distance: 0.4716981132075472
Raw accuracy top 5 euclidian distance: 0.35
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.4905660377358491
Raw accuracy top 5 cosine similarity: 0.37
Raw accuracy top 1 cosine similarity: 1.0

VGG16

Raw accuracy top 10 euclidian distance: 0.38679245283018865
Raw accuracy top 5 euclidian distance: 0.29
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.37735849056603776
Raw accuracy top 5 cosine similarity: 0.29500000000000004
Raw accuracy top 1 cosine similarity: 1.0

In [ ]:
# iterate over the queries
# set the number of retrieved images to display
num_retrievals = 5

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]

    # sort the distances in descending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_similarities = [x[1]*100 for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their distances
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"Similarity: {top_similarities[i]:.2f}%")
    plt.show()


# set the number of retrieved images to display

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

    # sort the distances in ascending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_distances = [x[1] for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their similarities
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"EuclidianD: {top_distances[i]:.2f}%")
    plt.show()

The cosine_similarity function returns a matrix of pairwise cosine similarities between the query embedding and the gallery embeddings. Since we are comparing one query image with multiple gallery images, we need to take the first element of the first dimension to get a single similarity score. Also, note that we pass reverse=True to sorted function to sort the distances in descending order since cosine similarity values range between -1 and 1, with higher values indicating more similarity.